A sentiment classifier that outperforms the current top HuggingFace leaderboard on the `rotten_tomatoes` sentiment dataset.

In [1]:
import sys
import os

notebook_dir = os.path.abspath('')
parent_dir = os.path.dirname(notebook_dir)
sys.path.insert(0, parent_dir)

In [2]:
# Auto reload modules
%load_ext autoreload
%autoreload 2

In [4]:
from dataset import Dataset, Embedder
from model import ClassificationModel
import json

In [14]:
# Load the config file
type = "sentiment_imdb"
# type = "sentiment_rottentomatoes"

with open (f'configs/{type}.json') as f:
    config = json.load(f)

In [15]:
# Create the test and validation datasets

# model = 'E5'
# model = 'openai'
model = 'InstructorXL'

embedder = Embedder(model)
config_dataset = config['datasets'][0]
dataset_train = Dataset(type, config_dataset, split='train')
dataset_test = Dataset(type, config_dataset, split='test')

load INSTRUCTOR_Transformer
max_seq_length  512


Found cached dataset imdb (/root/.cache/huggingface/datasets/imdb/plain_text/1.0.0/d613c88cf8fa3bab83b4ded3713f1f74830d1100e171db75bbddb80b3345c9c0)
Loading cached processed dataset at /root/.cache/huggingface/datasets/imdb/plain_text/1.0.0/d613c88cf8fa3bab83b4ded3713f1f74830d1100e171db75bbddb80b3345c9c0/cache-1b81e8ac53f6247f.arrow
Found cached dataset imdb (/root/.cache/huggingface/datasets/imdb/plain_text/1.0.0/d613c88cf8fa3bab83b4ded3713f1f74830d1100e171db75bbddb80b3345c9c0)
Loading cached processed dataset at /root/.cache/huggingface/datasets/imdb/plain_text/1.0.0/d613c88cf8fa3bab83b4ded3713f1f74830d1100e171db75bbddb80b3345c9c0/cache-4eaf59cde9752616.arrow


In [16]:
import torch

device = torch.device('cuda')
embedder.embed(dataset_train, batch_size=80, device=device, save=True)
embedder.embed(dataset_test, batch_size=80, device=device, save=True)

100%|██████████| 500/500 [28:23<00:00,  3.41s/it]


In [17]:
# Fit and test the linear model
config_model = config['model']
model = ClassificationModel(config_model)

result_train = model.fit(dataset_train)
print('Training Results', result_train)

result_test = model.test(dataset_test)
print('Validation Results:', result_test)

Training Results {'accuracy': 0.93976, 'f1': 0.9400954653937947}
Validation Results: {'accuracy': 0.93912, 'f1': 0.9395983808238749}


In [ ]:
path = f'models/{type}/model.npz'
model.save(path)
mode.toTorch(save=True, path=path)